## Описание бизнеса:
Букмекерская онлайн-контора «Пари лайн»

Необходимый анализ:
1. Сколько человеку надо прийти раз, чтобы сделать ставку?
2. Средний выигрыш на пользователя?
3. Баланс по каждому пользователю?
4. Самые выгодные города?
5. В каких городах самая высокая ставка?
6. Сколько в среднем времени занимает у человека с первого посещения сайта до первой попытки?

Импортирование бибилиотек 

In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

Откроем файл со ставками и посмотрим на него 

In [2]:
log_df = pd.read_csv('log.csv')
display(log_df)

,Запись пользователя № - user_919,[2019-01-01 14:06:51,Unnamed: 2,Unnamed: 3
0,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
1,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
2,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
4,Запись пользователя № - user_917,[2019-01-01 23:34:55,156789.0,NaN
...,...,...,...,...
994,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,NaN
995,#error,NaN,10054.0,29265.0
996,#error,NaN,10454.0,NaN
997,#error,NaN,1000.0,NaN


Обработаем данные 

In [3]:
log_df = pd.read_csv('log.csv', header = None) #убираем заголовок, так как он не читаем 
log_df.columns = ['user_id', 'time', 'bet', 'win'] #создаем новые заголовки
log_df['user_id'].nunique #видим, что столбец содержаит ошибки #error
new_log = log_df[log_df.user_id != '#error'] #удалим все значения с ошибкой


new_log['user_id'] = new_log.user_id.str.split(' - ').apply(lambda x: x[1]) #преобразуем поле user_id, оставив там только идентификатор пользователя
new_log['time'] = new_log['time'].apply(lambda x:x[1:])  #преобразуем поле time
new_log['time'] = pd.to_datetime(new_log['time'], format='%Y-%m-%d %H:%M:%S') #преобразуем тип поля time new_log['time'][0]
new_log['bet'] = new_log['bet'].fillna(0) #заполним пропуски в столбце bet  на 0
new_log['win'] = new_log['win'].fillna(0) #заполним пропуски в столбце win  на 0
new_log['net'] = new_log['win'] - new_log['bet'] #признак net, хранящий сумму выигрыша или пройгрыша с учетом ставки

display(new_log)

,user_id,time,bet,win,net
0,user_919,2019-01-01 14:06:51,0.0,0.0,0.0
1,user_973,2019-01-01 14:51:16,0.0,0.0,0.0
2,user_903,2019-01-01 16:31:16,0.0,0.0,0.0
3,user_954,2019-01-01 17:17:51,0.0,0.0,0.0
4,user_954,2019-01-01 21:31:18,0.0,0.0,0.0
...,...,...,...,...,...
991,user_965,2019-04-20 12:55:41,800.0,6927.0,6127.0
992,user_967,2019-04-20 14:59:36,10154.0,0.0,-10154.0
993,user_973,2019-04-20 17:09:56,10254.0,0.0,-10254.0
994,user_977,2019-04-20 18:10:07,10354.0,0.0,-10354.0


С данной таблицей мы можем ответить уже на следующие вопросы:
1. Сколько человеку надо прийти раз, чтобы сделать ставку?
2. Средний выигрыш на пользователя?
3. Баланс по каждому пользователю?
6. Сколько в среднем времени занимает у человека с первого посещения сайта до первой попытки?

Сколько человеку надо прийти раз, чтобы сделать ставку?

In [4]:
group1 = new_log[new_log.bet==0 ].groupby('user_id').bet.count() #группа клиентов, которые не делали ставку
group2 = new_log[new_log.bet!=0 ].groupby('user_id').bet.count() #группа клиентов, которые делали ставку
new_group = pd.merge(group1, group2, on='user_id') #объединяем в общий датафрейм
new_group.drop(new_group[new_group.bet_y == 0].index) #удаляем клиентов, которые так и не сделали ставку 
x = sum(new_group.bet_x)/len(new_group) 
print('Человеку в среднем надо прийти {:.0f} раз, чтобы сделать ставку'.format(x))

Человеку в среднем надо прийти 5 раз, чтобы сделать ставку


Средний выигрыш на пользователя?

In [5]:
net = new_log[new_log['net']>0] #создаем датафрейм, где учитываем только пройгрыши 
n = net['net'].mean()
print('Средний выигрыш на пользователя: {:.2f}'.format(n))

Средний выигрыш на пользователя: 82625.96


Баланс по каждому пользователю?

In [6]:
new_log.groupby('user_id')['net'].sum() #сгруппируем данные по user_id и посчитаем итоговый net для каждого 

user_id
user_1000     3101.0
user_900     20697.0
user_901      -600.0
user_902      2321.0
user_903     -6828.0
              ...   
user_995     -6965.0
user_996      3056.0
user_997     -1200.0
user_998     26195.0
user_999     -1600.0
Name: net, Length: 101, dtype: float64

Сколько в среднем времени занимает у человека с первого посещения сайта до первой попытки?

In [7]:
no_bet = new_log[new_log['bet'] == 0] #создаем датафрейм, где нет ставок
bet = new_log[new_log['bet'] != 0] #создаем датафрейм, где усть ставок

time1 = no_bet.groupby('user_id')['time'].min() #группируем по первому заходу
time2 = bet.groupby('user_id')['time'].min() #группируем по первому ставке

merge_time = pd.merge(time1,time2, on='user_id') #объединям данные
merge_time['delta'] = merge_time['time_y'] - merge_time['time_x'] #выводим разницу во времени 
display(merge_time)

,time_x,time_y,delta
user_id,,,
user_1000,2019-01-21 19:31:34,2019-01-28 20:57:37,7 days 01:26:03
user_900,2019-01-01 04:37:25,2019-02-08 10:36:47,38 days 05:59:22
user_901,2019-01-07 08:45:59,2019-03-14 13:51:20,66 days 05:05:21
user_902,2019-01-28 03:12:06,2019-03-09 16:52:11,40 days 13:40:05
user_903,2019-01-01 16:31:16,2019-02-08 11:06:00,37 days 18:34:44
...,...,...,...
user_995,2019-01-07 09:56:20,2019-02-26 15:29:25,50 days 05:33:05
user_996,2019-03-03 11:43:38,2019-01-10 07:32:17,-53 days +19:48:39
user_997,2019-01-05 20:55:29,2019-03-05 11:05:39,58 days 14:10:10


Удалим отрицательное значение и посчитаем среднее время 

In [8]:
merge_time.drop(merge_time.index[[97]])
t = merge_time['delta'].sum()/len(merge_time)
print('В среднем {} занимает у человека с первого посещения сайта до первой ставки'.format(t))

В среднем 46 days 01:16:27 занимает у человека с первого посещения сайта до первой ставки


Далее ответим на оставшиеся 2 вопроса:
1. Самые выгодные города?
2. В каких городах самая высокая ставка?

Откроем файл с городами 

In [9]:
user_df = pd.read_csv('users.csv')

Файл не читается, начнем его обработку

In [10]:
user_df = pd.read_csv('users.csv', sep ='\s', engine='python', encoding="koi8-r") #открываем файл с доп параметрами 
display(user_df)

,Юзверь,мейл,Гео
0,User_943,Accumanst@gmail.com,Ижевск
1,User_908,Advismowr@mail.ru,Ижевск
2,User_962,Anachso@ukr.net,Краснодар
3,User_973,Antecia@inbox.ru,Пермь
4,User_902,Balliaryva@ukr.net,None
...,...,...,...
95,User_959,UpdatesCurious@yahoo.com,Тюмень
96,User_901,V2artierso@mail.ru,Арзангелтск
97,User_970,Vashoterlo@bk.ru,Воронеж
98,User_965,Visuareda@yahoo.com,Воронеж


Продолжим обработку

In [11]:
user_df.columns = ['user_id', 'mail', 'geo']
user_df.user_id = user_df.user_id.apply(lambda x: x.lower()) 
display(user_df)

,user_id,mail,geo
0,user_943,Accumanst@gmail.com,Ижевск
1,user_908,Advismowr@mail.ru,Ижевск
2,user_962,Anachso@ukr.net,Краснодар
3,user_973,Antecia@inbox.ru,Пермь
4,user_902,Balliaryva@ukr.net,None
...,...,...,...
95,user_959,UpdatesCurious@yahoo.com,Тюмень
96,user_901,V2artierso@mail.ru,Арзангелтск
97,user_970,Vashoterlo@bk.ru,Воронеж
98,user_965,Visuareda@yahoo.com,Воронеж


Объединим отформатированные таблицы

In [12]:
merge_df = pd.merge(new_log, user_df, on = 'user_id')
display(merge_df)

,user_id,time,bet,win,net,mail,geo
0,user_919,2019-01-01 14:06:51,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
1,user_919,2019-01-30 10:06:00,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
2,user_919,2019-02-05 14:33:44,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
3,user_919,2019-02-14 11:38:05,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
4,user_919,2019-03-02 04:23:36,300.0,0.0,-300.0,Chikkaverle@icloud.com,Хабаровск
...,...,...,...,...,...,...,...
970,user_932,2019-02-24 22:40:06,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
971,user_932,2019-03-15 10:56:14,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
972,user_932,2019-03-18 10:13:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
973,user_932,2019-03-27 12:18:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск


Самые выгодные города?

In [18]:
merge_df.groupby('geo')['user_id'].count().sort_values(ascending=False) #выделим города по кол-ву ставок 


geo
Санкт-Петербург    115
Арзангелтск         96
Ярославль           89
Воронеж             88
Краснодар           86
Казань              66
Москва              61
Ижевск              61
Хабаровск           60
Красноярск          56
Пермь               55
Екатеринбург        49
Ставрополь          36
Тюмень              32
Name: user_id, dtype: int64

В каких городах самая высокая ставка?

In [14]:
merge_df.groupby('geo')['bet'].mean().sort_values(ascending=False) #выделим города по среднему значению 

geo
Москва             39793.327869
Ставрополь          1582.500000
Красноярск          1428.178571
Воронеж             1324.840909
Пермь               1135.818182
Ижевск              1079.836066
Краснодар            839.813953
Хабаровск            757.700000
Ярославль            750.000000
Казань               714.575758
Арзангелтск          663.708333
Санкт-Петербург      654.521739
Екатеринбург         228.571429
Тюмень               175.000000
Name: bet, dtype: float64

In [15]:
merge_df.groupby('geo')['bet'].max().sort_values(ascending=False) #выделим города по максимальному значению  

geo
Москва             1678321.0
Хабаровск            10754.0
Санкт-Петербург      10754.0
Пермь                10754.0
Воронеж              10754.0
Казань               10654.0
Ижевск               10654.0
Ярославль            10554.0
Красноярск           10554.0
Арзангелтск          10554.0
Краснодар            10354.0
Ставрополь           10254.0
Тюмень                1000.0
Екатеринбург           900.0
Name: bet, dtype: float64